# Capstone Design: Sedimentation Tank Remainder

### Team SYM

### Shiyao Sun and Melanie Lim

### Monroe's comments 
(include this in your next submission so I can verify that you've addressed all of these issues.)
<font color="red">  </font>
* <font color="red">  Add an introduction explaining the problem that you are solving. The goal is to design a sed tank that can be used in many communities.
* Did you review the mathcad file that was used as the previous design? I get the impression that you didn't use that very valuable resource.</font> 
* Define the problem and explain how you will solve it.</font>
* Set this up for working in Python. Define the constants that you already know. You can look in the expert inputs file (see the mathcad source files linked below) to see if there are any other constraints you are missing for the sed tank.
* Start with the python code from the sed tank design challenge.
For the next submission you will want to have the majority of the sed tank designed. This is a significant challenge. The AIDE team can help specify what level of detail they need to be able to draw the sed tank.

It seems that you missed the guidance for this report. See https://confluence.cornell.edu/display/cee4540/DC+Capstone+Design
well documented report created in Jupyter with responses to comments received from previous submissions.
The report must be formatted to be one page wide and must be organized with text used to introduce and explain equations, images, and design methods.
Explain ALL calculations step by step including why you are doing the calculation.
Present each equation you use in a markdown cell using beautiful Latex and then follow that with the code cell with the function written in Python.
If you need to display results in a table, use a DataFrame to make a nicely formatted result that changes with input parameters.
The report must explain why you have made the decisions that you have made.
Explain the physics of the problem and how that influences your solution scheme.
Explain the constraints 
Document your sources of information with in text citations and a bibliography.
Relevant equations created using Latex and included with the text of the report.
Jupyter worksheet must be organized as a stand alone report that can be read and easily understood from beginning to end.
Include sketches, drawings, graphs as part of the Jupyter document. Either include any required images and zip all of the files together for submission.
Include sketches that show the geometry of your system and make sure that all variables are clearly defined. Write in paragraph format so that your work is easy to understand.
Clearly explain which equations you developed and which equations you borrowed from the AguaClara source code.
Spell check before submitting and proofread the entire report.

In [1]:
from aide_design.play import*
from aide_design import floc_model as floc
from pytexit import py2tex
import math
from aide_design import expert_inputs as exp

<div class="alert alert-block alert-info">

# <u>Introduction</u>

The goal of this Capstone Design challenge is to design a sedimentation tank that can be used in multiple communities. According to given inputs focused on the remainder of the sedimentation tank, the tank dimensions can be calculated using flow equations and functions we have learned throughout this semester.


## Specify all the assumptions

In [2]:
# We are experimenting with using head loss at the exit of the diffusers as
# a reasonable design constraint for the design of the
# inlet manifold/diffuser system. This head loss in the exit of the diffuser
# will allow higher velocities (and pressure recovery) in the manifold pipe
# and thus will enable use of smaller diameter manifold pipes.
# -----
# We've set a maximum head loss in the sedimentation tank inlet:
headloss_sed_inlet_max = 1 * u.cm

# The diffuser pipes used in the sedimentation tank have an SDR of 26.
# The manifold used in the sedimentation tank has an SDR of 41.
SDR_diffuser = 26
SDR_manifold = 41

# Pi_jet_plane is used to estimate the maximum energy dissipation rate in a plane jet. 
# A plane jet is uniform in one dimension. A plane jet can be formed by a long narrow slot.
Pi_jet_plane = 0.225

# The PVC pipe used to make the inlet manifold
L_sed = 6 * u.m

# Design temperature will be helpful to calculate viscosity
T_design = 15 * u.degC

# Upflow velocity at the top of the floc blanket:
V_sed_up = 1 * u.mm/u.s

# The corrugated plastic sheets used to make the plate settlers:
W_sed = (42 * u.inch).to(u.m)
thickness_sed_plate = 2 * u.mm

# The plate settlers are angled 60° from the horizontal:
angle_sed_plate = 60 * u.deg

# The plate setters are spaced 2.5cm apart (this is the perpendicular
# distance between plates, not the horizontal distance between plates):
s_sed_plate = 2.5 * u.cm

# Plate settler capture velocity:
V_sed_capture = 0.12 * u.mm/u.s

# The minimum port flow (from the first port) divided by the maximum port 
# flow (from the last port) for flow division between sedimentation tanks 
# and for flow distribution from the inlet manifold should be at least:
Q_ratio = 0.8

# Assumed stretch of the PVC pipes as they are heated and molded:
Pi_PVC_stretch = 1.2

# Nominal diameter of the sed tank diffuser
ND_sed_diffuser = 1 * u.inch

# Assume the minor loss coefficient to be 1
K = 1

# Assume headloss exiting the tank is 5 cm
headloss_exit = 5 * u.cm

# Assume Vena contract (VC) is 0.63
VC = exp.RATIO_VC_ORIFICE

# Assume spacing between orifices in outlet manifold is 10cm
spacing_orifices = 10 * u.cm

g = pc.gravity

## Inlet Diffusers

We need one function to calculate the width of diffuser and one function to calculate the length of the diffuser.
<font color="red"> I'm not sure what constraint sets this length. Shorter is stronger. Longer does a better job of creating a uniform velocity from the diffuser tip, but this likely isn't very significant. For now I'd suggest using a length of 6 inches.

You have equations below, but you don't use them for anything. This is true throughout the document. Why aren't there any calculations or design results? You have a design challenge where you designed inlet diffusers. Why not use that as a starting point?
 </font>

$$Vel_{Diffuser,Max}=\left(\frac{2\,Headloss_{SedInlet,Max}\,g}{K_{Minor}}\right)^{\frac{1}{2}}$$

In [3]:
# Find the pipe's outer diameter according to its nominal diameter
OD_sed_diffuser = pipe.OD(ND_sed_diffuser)

# Find the pipe's inner diameter
ID_sed_diffuser = pipe.ID_SDR(ND_sed_diffuser,SDR_diffuser)

# Find the area of the PVC
Area_pvc = np.pi * (OD_sed_diffuser/2)**2 - np.pi * (ID_sed_diffuser/2)**2

# Find the thickness of the diffuser
thickness_wall = 1 / Pi_PVC_stretch * (OD_sed_diffuser - ID_sed_diffuser) /2 

def Width_diffuser (W_sed, V_sed_up, max_headloss, minorloss_coefficient):
    MaxV_diffuser = np.sqrt (2 * g * max_headloss / minorloss_coefficient)
    MinInnerW_diffuser = (V_sed_up * W_sed / MaxV_diffuser).to(u.inch)
    Available_thickness = np.arange(1/16, MinInnerW_diffuser.magnitude + 1/16 ,1/16)
    Width_diffuser = (ut.ceil_nearest(MinInnerW_diffuser.magnitude,Available_thickness))*u.inch
    return (Width_diffuser)

Width_diffuser = Width_diffuser (W_sed, V_sed_up, headloss_sed_inlet_max,K)
print('The width of the diffuser is ' + ut.sig(Width_diffuser.to(u.cm),4) +'.')

# Find the outer length of the rectangular diffuser slot
B_diffuser = Area_pvc / thickness_wall / 2 - Width_diffuser

# Find the inner length of the rectangular diffuser slot
S_diffuser = B_diffuser - 2 * thickness_wall
Q_diffuser = V_sed_up * W_sed * B_diffuser
V_diffuser = Q_diffuser / Width_diffuser / S_diffuser



The width of the diffuser is 0.3175 cm.


In [4]:
def Length_diffuser

SyntaxError: invalid syntax (<ipython-input-4-40d55aef25b5>, line 1)

## Inlet Manifold

Equations and Sketch

$$\left(\frac{1 - {\Pi_{DiffuserFlow}}^{2}}{{\Pi_{DiffuserFlow}}^{2} + 1} \right) {hl}_{ParallelPath} = \frac{{{Velocity}_{Manifold}}^{2}}{4g}$$

In [ ]:
def ND_inlet_manifold(L_sed, W_sed, V_Upflow, V_diffuser, SDR_manifold, Q_ratio):
    hl = V_diffuser **2 / 2 / pc.gravity
    max_velocity =  ((1-Q_ratio **2)/(Q_ratio**2+1) * hl * 4 * pc.gravity) ** (1/2) 
    Q_sed = L_sed * W_sed * V_Upflow
    Min_ID_manifold =((Q_sed / max_velocity) / math.pi * 4) ** (1/2)
    return (pipe.ND_SDR_available(Min_ID_manifold, SDR_manifold))

ND_inlet_manifold = ND_inlet_manifold(L_sed, W_sed, V_sed_up, V_diffuser, SDR_manifold, Q_ratio)
print('The nominal diameter of the inlet manifold is ' + ut.sig(ND_inlet_manifold.to(u.cm),4) +'.')

## Plate Settlers

$$L = \frac{{S\left( {\frac{{{V_{Plate \uparrow }}}}{{{V_c}}} - 1} \right)+ T \frac{V_{Plate \uparrow }}{V_C} }} {{\sin \alpha \cos \alpha }}$$

In [ ]:
def Len_sed_plate(V_Upflow, V_Capture, Spacing_Settlers, Angle_sed_plate, Thickness_sed_plate):
    # Return the length of the plate settlers
    Top = Spacing_Settlers * (V_Upflow / V_Capture -1) + Thickness_sed_plate * (V_Upflow / V_Capture)
    Bottom = math.cos(Angle_sed_plate) * math.sin(Angle_sed_plate)
    Len_sed_plate = Top / Bottom
    return Len_sed_plate.to(u.cm)

def Vertical_Height_plate(V_Upflow, V_Capture, Spacing_Settlers, Angle_sed_plate, Thickness_sed_plate):
    # Return the vertical length of the plate settlers
    len_sed_plate = Len_sed_plate(V_Upflow, V_Capture, Spacing_Settlers, Angle_sed_plate, Thickness_sed_plate)
    return len_sed_plate * math.sin(Angle_sed_plate)

sed_plate_length = Len_sed_plate(V_sed_up, V_sed_capture, s_sed_plate, angle_sed_plate, thickness_sed_plate)
sed_plate_vertical_height = Vertical_Height_plate(V_sed_up, V_sed_capture, s_sed_plate, angle_sed_plate, thickness_sed_plate)

print('The required length of the plate settlers is ' + ut.sig(sed_plate_length.to(u.cm),4) +'.')
print('The vertical height of the plate settlers is ' + ut.sig(sed_plate_vertical_height.to(u.cm),4) + '.')


<br>

* <font color="red"> Include definition sketch showing dimensions with variable names.
* Calculate the effect of the wasted space at the end of the plate settlers.
* Show the equations that you use in Latex. Perhaps don't need all equations, but key equations should be shown.
</font>


## Outlet Manifold (Effluent Launder)

* (Diameter of) Outlet manifold (Effluent Launder) same inputs as Inlet manifold
* Function of (L_sed, W_Sed, V_up, Q_ratio, h_L_flowcontrol)
* Headloss flow control (for water leaving the sed tank) = 5 cm
* Outlet manifold system^
* Big enough so that the decrease in speed through inlet channel doesn’t matter



In [ ]:
def ND_outlet_manifold(L_sed, W_sed, V_Upflow, SDR_manifold, Q_ratio, headloss_exit):
    max_velocity =  ((1-Q_ratio **2)/(Q_ratio**2+1) * headloss_exit * 4 * pc.gravity) ** (1/2) 
    Q_sed = L_sed * W_sed * V_Upflow
    Min_ID_manifold =((Q_sed / max_velocity) / math.pi * 4) ** (1/2)
    return (pipe.ND_SDR_available(Min_ID_manifold, SDR_manifold))

ND_outlet_manifold = ND_outlet_manifold(L_sed, W_sed, V_sed_up, SDR_manifold, Q_ratio,headloss_exit)
print('The nominal diameter of the outlet manifold is ' + ut.sig(ND_outlet_manifold.to(u.cm),4) +'.')
    

## Orifices 

* Orifices a function of (spacing between holes, headloss_orifice)
* Headloss_orifice = headloss_total - h_L_manifold ← changed his mind
* Headloss_orifice = 4 cm
* Spacing = 10 cm


In [ ]:
def Diam_Orifices (L_sed, W_sed, V_Upflow, spacing_orifices, headloss_exit, headloss_sed_inlet_max, RatioVCorifice):
    headloss_orifice = headloss_exit - headloss_sed_inlet_max
    num_orifices = L_sed / spacing_orifices
    headloss_perorifice = headloss / num_orifices
    Q_sed = L_sed * W_sed * V_Upflow
    Area_orifice = Q_sed / (RatioVCorifice * (headloss_perorifice* 2* g)**(1/2))
    return (4 / pi * Area_orifice) **(1/2)

Diam_Orifices = Diam_Orifices (L_sed, W_sed, V_sed_up,spacing_orifices, headloss_exit, headloss_sed_inlet_max, VC)
print('The diameter of the orifice is '+ut.sig(Diam_Orifices.to(u.cm),4)+'.')

    
    